# SOFC-IES Report Basic Fixed Price Analysis

In [1]:
import os
import pyomo.environ as pyo
import idaes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

from idaes.core.surrogate.alamopy import AlamoTrainer
from idaes.core.surrogate.sampling import split_training_validation
from idaes.core.surrogate.metrics import compute_fit_metrics
from idaes.core.surrogate.plotting.sm_plotter import surrogate_residual, surrogate_parity
from idaes.core.surrogate.alamopy import AlamoSurrogate
from idaes.core.surrogate.surrogate_block import SurrogateBlock

from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

solver_obj = pyo.SolverFactory("ipopt")

## Set Parameters

In [2]:
do_contours = False
# contour plot resolution
no_samples = 30

# Fixed costs $/hr
model0_fixed_cap = 128.66e6/365/24
model0_fixed_om = 43.88e6/365/24
model0_fixed_cost = model0_fixed_cap + model0_fixed_om

model1_fixed_cap = 70.37*1e6/365/24
model1_fixed_om = 49.53*1e6/365/24
model1_fixed_cost = model1_fixed_cap + model1_fixed_om

model3_fixed_cap = 145.61e6/365/24
model3_fixed_om = 72.73e6/365/24
model3_fixed_cost = model3_fixed_cap + model3_fixed_om

model4_fixed_cap = 73.65e6/365/24
model4_fixed_om = 51.69e6/365/24
model4_fixed_cost = model4_fixed_cap + model4_fixed_om

model5_fixed_cap = 85.24*1e6/365/24
model5_fixed_om = 62.65*1e6/365/24
model5_fixed_cost = model5_fixed_cap + model5_fixed_om

model6_fixed_cap = 49.38*1e6/365/24
model6_fixed_om = 29.92*1e6/365/24
model6_fixed_cost = model6_fixed_cap + model6_fixed_om

## Surrogate Models

In this section surrogate models for variable costs are fitted to data from the models in 000_SOFC_IES_data.xlsx.

### Surrogate Generation Function

In [3]:
# Surrogate Model Generation
# IDAES API
def train_SOFC_costing(fname='000_SOFC_IES_data.xlsx', case='ngcc', plot=False, input_labels=None, output_labels=None):
# Read in the data from csv file
    df = pd.read_excel(fname, sheet_name=case)
    # Training and Validation Data
    n_data = df[input_labels[0]].size
    # Split the training and validation data
    df_training, df_validation = split_training_validation(df, 0.8, seed=n_data)  # seed=1975)

    if not os.path.exists(str(case)+"_alamo_surrogate.json"):
        print('\n ------------------------------------------------- \n')
        print('Fitting surrogate model')
        print('\n ------------------------------------------------- \n')
        # Create ALAMO trainer object
        trainer = AlamoTrainer(
            input_labels=input_labels, 
            output_labels=output_labels,
            training_dataframe=df_training,
        )
        # Set ALAMO options
        trainer.config.linfcns = True
        trainer.config.constant = True
        trainer.config.monomialpower = [2, 3]
        trainer.config.multi2power = [1, 2, 3]
        trainer.config.multi2power = [1, 2, 3]
        trainer.config.ratiopower = [1, 2]
        trainer.config.maxtime = 2500.0
        trainer.config.maxterms = 8, 8, 8, 8
        # Train surrogate (this will call to Alamo through AlamoPy)
        success, alamo_surrogate, msg = trainer.train_surrogate()
        # Display the metrics of fit
        metrics = compute_fit_metrics(alamo_surrogate, df_validation)
        surrogate_parity(alamo_surrogate, df_training, filename=str(case)+'_parity_plots.pdf')
        # Save the alamo surrogate
        alamo_surrogate.save_to_file(str(case)+'_alamo_surrogate.json', overwrite=True)

    else:
        # Load the surrogate object
        alamo_surrogate = AlamoSurrogate.load_from_file(str(case)+'_alamo_surrogate.json')
        print('\n ------------------------------------------------- \n')
        print('Loading surrogate model from json file')
        print('\n ------------------------------------------------- \n')
        # Display the metrics of fit
        metrics = compute_fit_metrics(alamo_surrogate, df_validation)
        surrogate_parity(alamo_surrogate, df_training, filename=str(case)+'_parity_plots.pdf')
    return alamo_surrogate, metrics

### Case 0 -- NGCC

In [4]:
alamo_surrogate0, metrics0 = train_SOFC_costing(
    case="ngcc", 
    plot=False, 
    input_labels=["net_power"],
    output_labels=["total_var_cost", "fuel_var_cost", "other_var_cost"],
)
for o in metrics0:
    print(f"\n{o}")
    for m in metrics0[o]:
        print(f"    {m} {metrics0[o][m]}")
    print(f"   {alamo_surrogate0._surrogate_expressions[o]}")

FileNotFoundError: [Errno 2] No such file or directory: '000_SOFC_IES_data.xlsx'

### Case 1 -- SOFC

In [ ]:
alamo_surrogate1, metrics1 = train_SOFC_costing(
    case="sofc", 
    plot=False, 
    input_labels=["net_power"],
    output_labels=["total_var_cost", "fuel_var_cost", "other_var_cost"],
)
for o in metrics1:
    print(f"\n{o}")
    for m in metrics1[o]:
        print(f"    {m} {metrics1[o][m]}")
    print(f"   {alamo_surrogate1._surrogate_expressions[o]}")

### Case 3 -- NGCC + SOFC

In [ ]:
# The NGCC + SOEC with the SOEC off uses the case 0 (NGCC) surrogate
alamo_surrogate3p = alamo_surrogate0
metrics3p = metrics0

In [ ]:
alamo_surrogate3, metrics3 = train_SOFC_costing(
    case="ngcc_soec", 
    plot=False, 
    input_labels=["net_power", "h_prod"],
    output_labels=["total_var_cost", "fuel_var_cost", "other_var_cost"],
)
for o in metrics3:
    print(f"\n{o}")
    for m in metrics3[o]:
        print(f"    {m} {metrics3[o][m]}")
    print(f"   {alamo_surrogate3._surrogate_expressions[o]}")

### Case 4 -- rSOFC

In [ ]:
# The rSOFC SOFC mode variable cost surrogate is the same as the SOFC
alamo_surrogate4a = alamo_surrogate1
metrics4a = metrics1

In [ ]:
alamo_surrogate4b, metrics4b = train_SOFC_costing(
    case="rsofc_soec", 
    plot=False, 
    input_labels=["h_prod"],
    output_labels=["total_var_cost", "elec_var_cost", "fuel_var_cost", "other_var_cost"],
)
for o in metrics4b:
    print(f"\n{o}")
    for m in metrics4b[o]:
        print(f"    {m} {metrics4b[o][m]}")
    print(f"   {alamo_surrogate4b._surrogate_expressions[o]}")

### Case 5 -- SOFC + SOEC

#### Power + Hydrogen

In [ ]:
alamo_surrogate5, metrics5 = train_SOFC_costing(
    case="sofc_soec", 
    plot=False, 
    input_labels=["net_power", "h_prod"],
    output_labels=["total_var_cost", "fuel_var_cost", "other_var_cost"],
)
for o in metrics5:
    print(f"\n{o}")
    for m in metrics5[o]:
        print(f"    {m} {metrics5[o][m]}")
    print(f"   {alamo_surrogate5._surrogate_expressions[o]}")

#### Power Only

In [ ]:
alamo_surrogate5p, metrics5p = train_SOFC_costing(
    case="sofc_soec_power_only", 
    plot=False, 
    input_labels=["net_power"],
    output_labels=["total_var_cost", "fuel_var_cost", "other_var_cost"],
)
for o in metrics5p:
    print(f"\n{o}")
    for m in metrics5p[o]:
        print(f"    {m} {metrics5p[o][m]}")
    print(f"   {alamo_surrogate5p._surrogate_expressions[o]}")

### Caes 6 -- SOEC

In [ ]:
alamo_surrogate6, metrics6 = train_SOFC_costing(
    case="soec", 
    plot=False, 
    input_labels=["h_prod"],
    output_labels=["total_var_cost", "elec_var_cost", "other_var_cost"],
)
for o in metrics6:
    print(f"\n{o}")
    for m in metrics6[o]:
        print(f"    {m} {metrics6[o][m]}")
    print(f"   {alamo_surrogate6._surrogate_expressions[o]}")

## Models

### Case 0 -- NGCC

In [ ]:
def get_model0():

    model0 = pyo.ConcreteModel(name="NGCC")

    model0.profit = pyo.Var()
    model0.fuel_cost = pyo.Var()
    model0.other_cost = pyo.Var()
    model0.total_cost = pyo.Var()

    model0.fixed_costs = pyo.Var(initialize=model0_fixed_cost) #All fixed costs $/hr
    model0.fixed_costs.fix()

    model0.ng_price = pyo.Var(initialize=4.42) # Natural gas price $/million BTU
    model0.ng_price.fix()

    model0.el_price = pyo.Var(initialize=71.70) # Electricity price $/MWhr
    model0.el_price.fix()
    model0.net_power = pyo.Var(initialize=650) # net power mw

    # import/build surrogate block (this will also set net power bounds)
    model0.surrogate = SurrogateBlock()
    model0.surrogate.build_model(
        alamo_surrogate0,
        input_vars=[model0.net_power],
        output_vars=[
            model0.total_cost,
            model0.fuel_cost,
            model0.other_cost,
        ]
    )

    model0.profit_eqn = pyo.Constraint(
        expr=model0.profit ==
        model0.el_price*model0.net_power - 
        model0.fixed_costs - 
        (model0.ng_price/4.42)*model0.fuel_cost - 
        model0.other_cost
    )

    model0.obj = pyo.Objective(expr=-model0.profit)
    
    return model0

### Case 1 -- SOFC

In [ ]:
def get_model1():
    
    model1 = pyo.ConcreteModel(name="rSOFC_SOFC_mode")

    model1.profit = pyo.Var()
    model1.fuel_cost = pyo.Var()
    model1.other_cost = pyo.Var()
    model1.total_cost = pyo.Var()

    model1.fixed_costs = pyo.Var(initialize=model1_fixed_cost) #All fixed costs $/hr
    model1.fixed_costs.fix()
    model1.ng_price = pyo.Var(initialize=4.42) # Natural gas price $/million BTU
    model1.ng_price.fix()
    model1.el_price = pyo.Var(initialize=71.70) # Electricity price $/MWhr
    model1.el_price.fix()
    model1.net_power = pyo.Var(initialize=650) # net power mw

    # import/build surrogate block (Sets net power bounds)
    model1.surrogate = SurrogateBlock()
    model1.surrogate.build_model(
        alamo_surrogate1,
        input_vars=[model1.net_power],
        output_vars=[
            model1.total_cost,
            model1.fuel_cost,
            model1.other_cost,
        ]
    )

    model1.profit_eqn = pyo.Constraint(
        expr=model1.profit ==
        model1.el_price*model1.net_power - 
        model1.fixed_costs - 
        (model1.ng_price/4.42)*model1.fuel_cost - 
        model1.other_cost
    )

    model1.obj = pyo.Objective(expr=-model1.profit)
    
    return model1

### Case 3 -- NGCC + SOFC

In [ ]:
def get_model3p():
    # Power only
    model3p = get_model0()
    model3p.fixed_costs.fix(model3_fixed_cost)
    return model3p

In [ ]:
def get_model3():
    # Power and hydrogen
    model3 = pyo.ConcreteModel(name="NGCC + SOEC")

    model3.profit = pyo.Var()
    model3.fuel_cost = pyo.Var()
    model3.other_cost = pyo.Var()
    model3.total_cost = pyo.Var()

    model3.fixed_costs = pyo.Var(initialize=model3_fixed_cost) #All fixed costs $/hr
    model3.fixed_costs.fix()
    model3.ng_price = pyo.Var(initialize=4.42) # Natural gas price $/million BTU
    model3.ng_price.fix()
    model3.el_price = pyo.Var(initialize=100) # Electricity price $/MWhr
    model3.el_price.fix()
    model3.h2_price = pyo.Var(initialize=2)
    model3.h2_price.fix()
    model3.net_power = pyo.Var(initialize=650) # net power mw
    model3.h2_prod = pyo.Var(initialize=0.75, bounds=(0.0, 5))

    # import/build surrogate block
    model3.surrogate = SurrogateBlock()
    model3.surrogate.build_model(
        alamo_surrogate3,
        input_vars=[model3.net_power, model3.h2_prod],
        output_vars=[
            model3.total_cost,
            model3.fuel_cost,
            model3.other_cost,
        ]
    )

    model3.profit_eqn = pyo.Constraint(expr=model3.profit ==
        (
            model3.el_price*model3.net_power + 
            model3.h2_price*model3.h2_prod*3600 - 
            model3.fixed_costs - 
            (model3.ng_price/4.42)*model3.fuel_cost - 
            model3.other_cost
        )
    )
    model3.h2_prod.setub(5.0)

    model3.net_power_ineq1 = pyo.Constraint(expr=model3.net_power >=
        15.849*model3.h2_prod - 130.94
    )
    model3.net_power_ineq2 = pyo.Constraint(expr=model3.net_power >=
        -143.31*model3.h2_prod + 447.77
    )
    model3.net_power_ineq3 = pyo.Constraint(expr=model3.net_power <=
        -141.04*model3.h2_prod + 655.41
    )

    model3.obj = pyo.Objective(expr=-model3.profit)
    
    return model3

### Case 4 -- rSOFC

In [ ]:
def get_model4a():
    # Power Mode
    model4a = get_model1()
    model4a.fixed_costs.fix(model4_fixed_cost)
    return model4a

In [ ]:
def get_model4b():
    model4b = pyo.ConcreteModel(name="rSOFC_SOEC_mode")

    model4b.profit = pyo.Var()
    model4b.electricity_cost = pyo.Var()  # $/hr
    model4b.fuel_cost = pyo.Var()
    model4b.other_cost = pyo.Var()
    model4b.total_cost = pyo.Var()

    model4b.fixed_costs = pyo.Var(initialize=model4_fixed_cost) #All fixed costs $/hr
    model4b.fixed_costs.fix()
    model4b.ng_price = pyo.Var(initialize=4.42) # Natural gas price $/million BTU
    model4b.ng_price.fix()
    model4b.el_price = pyo.Var(initialize=100) # Electricity price $/MWhr
    model4b.el_price.fix()
    model4b.h2_price = pyo.Var(initialize=2)
    model4b.h2_price.fix()
    model4b.h2_prod = pyo.Var(initialize=0.75)

    # import/build surrogate block
    model4b.surrogate = SurrogateBlock()
    model4b.surrogate.build_model(
        alamo_surrogate4b,
        input_vars=[model4b.h2_prod],
        output_vars=[
            model4b.total_cost,
            model4b.electricity_cost,
            model4b.fuel_cost,
            model4b.other_cost,
        ]
    )
    model4b.h2_prod.setub(5.0)

    model4b.profit_eqn = pyo.Constraint(
        expr=model4b.profit ==
        model4b.h2_price*model4b.h2_prod*3600 - 
        (model4b.el_price/30)*model4b.electricity_cost -
        model4b.fixed_costs - 
        (model4b.ng_price/4.42)*model4b.fuel_cost - 
        model4b.other_cost
    )

    model4b.obj = pyo.Objective(expr=-model4b.profit)
    return model4b

### Case 5 -- SOFC + SOEC

In [ ]:
def get_model5p():
    # Power Mode
    model5p = pyo.ConcreteModel(name="SOFC_SOEC")

    model5p.profit = pyo.Var()
    model5p.fuel_cost = pyo.Var()
    model5p.other_cost = pyo.Var()
    model5p.total_cost = pyo.Var()

    model5p.fixed_costs = pyo.Var(initialize=model5_fixed_cost) #All fixed costs $/hr
    model5p.fixed_costs.fix()
    model5p.ng_price = pyo.Var(initialize=4.42) # Natural gas price $/million BTU
    model5p.ng_price.fix()
    model5p.el_price = pyo.Var(initialize=71.70) # Electricity price $/MWhr
    model5p.el_price.fix()
    model5p.net_power = pyo.Var(initialize=710) # net power mw
    
    # import/build surrogate block (Sets net power bounds)
    model5p.surrogate = SurrogateBlock()
    model5p.surrogate.build_model(
        alamo_surrogate5p,
        input_vars=[model5p.net_power],
        output_vars=[
            model5p.total_cost,
            model5p.fuel_cost,
            model5p.other_cost,
        ]
    )

    model5p.profit_eqn = pyo.Constraint(
        expr=model5p.profit ==
        model5p.el_price*model5p.net_power - 
        model5p.fixed_costs - 
        (model5p.ng_price/4.42)*model5p.fuel_cost - 
        model5p.other_cost
    )

    model5p.obj = pyo.Objective(expr=-model5p.profit)
    return model5p

In [ ]:
def get_model5():
    model5 = pyo.ConcreteModel(name="SOFC + SOEC")

    model5.profit = pyo.Var()
    model5.fuel_cost = pyo.Var()
    model5.other_cost = pyo.Var()
    model5.total_cost = pyo.Var()

    model5.fixed_costs = pyo.Var(initialize=model5_fixed_cost) #All fixed costs $/hr
    model5.fixed_costs.fix()
    model5.ng_price = pyo.Var(initialize=4.42) # Natural gas price $/million BTU
    model5.ng_price.fix()
    model5.el_price = pyo.Var(initialize=100) # Electricity price $/MWhr
    model5.el_price.fix()
    model5.h2_price = pyo.Var(initialize=2)
    model5.h2_price.fix()
    model5.net_power = pyo.Var(initialize=500) # net power mw
    model5.h2_prod = pyo.Var(initialize=0.75, bounds=(0.75, 5.0))

    # import/build surrogate block
    model5.surrogate = SurrogateBlock()
    model5.surrogate.build_model(
        alamo_surrogate5,
        input_vars=[model5.net_power, model5.h2_prod],
        output_vars=[
            model5.total_cost,
            model5.fuel_cost,
            model5.other_cost,
        ]
    )

    model5.profit_eqn = pyo.Constraint(
        expr=model5.profit ==
        model5.el_price*model5.net_power + 
        model5.h2_price*model5.h2_prod*3600 - 
        model5.fixed_costs - 
        (model5.ng_price/4.42)*model5.fuel_cost - 
        model5.other_cost
    )

    model5.net_power_ineq1 = pyo.Constraint(expr=model5.net_power <=
        -142.06*model5.h2_prod + 711.8
    )
    model5.net_power_ineq2 = pyo.Constraint(expr=model5.net_power >=
        -120.46*model5.h2_prod + 220.31
    )

    model5.obj = pyo.Objective(expr=-model5.profit)
    return model5

### Case 6 -- SOEC

In [ ]:
def get_model6():
    model6 = pyo.ConcreteModel(name="SOEC")

    model6.profit = pyo.Var()
    model6.electricity_cost = pyo.Var()  # $/hr
    model6.other_cost = pyo.Var()
    model6.total_cost = pyo.Var()

    model6.fixed_costs = pyo.Var(initialize=model6_fixed_cost) #All fixed costs $/hr
    model6.fixed_costs.fix()
    model6.el_price = pyo.Var(initialize=100) # Electricity price $/MWhr
    model6.el_price.fix()
    model6.h2_price = pyo.Var(initialize=2)
    model6.h2_price.fix()
    model6.h2_prod = pyo.Var(initialize=0.75)

    # import/build surrogate block
    model6.surrogate = SurrogateBlock()
    model6.surrogate.build_model(
        alamo_surrogate6,
        input_vars=[model6.h2_prod],
        output_vars=[
            model6.total_cost,
            model6.electricity_cost,
            model6.other_cost,
        ]
    )

    model6.profit_eqn = pyo.Constraint(
        expr=model6.profit ==
        model6.h2_price*model6.h2_prod*3600 - 
        (model6.el_price/30)*model6.electricity_cost -
        model6.other_cost  - 
        model6.fixed_costs
    )

    model6.obj = pyo.Objective(expr=-model6.profit)
    return model6

In [ ]:
model0 = get_model0()
model1 = get_model1()
model3 = get_model3()
model3p = get_model3p()
model4a = get_model4a()
model4b = get_model4b()
model5 = get_model5()
model5p = get_model5p()
model6 = get_model6()

## Bar Chart Compare

### Power Only

In [ ]:
model0.profit.unfix()
model0.net_power.fix(650)
model0.ng_price.fix(4.42)
model0.el_price.fix(71.70)

model1.profit.unfix()
model1.net_power.fix(650)
model1.ng_price.fix(4.42)
model1.el_price.fix(71.70)

model3p.profit.unfix()
model3p.net_power.fix(650)
model3p.ng_price.fix(4.42)
model3p.el_price.fix(71.70)

model4a.profit.unfix()
model4a.net_power.fix(650)
model4a.ng_price.fix(4.42)
model4a.el_price.fix(71.70)

model5p.profit.unfix()
model5p.net_power.fix(712)
model5p.ng_price.fix(4.42)
model5p.el_price.fix(71.70)


In [ ]:
cases = ["NGCC", "SOFC", "NGCC+SOEC", "rSOC", "SOFC+SOEC"]
capital = {}
fixed_om = {}
fuel = {}
other = {}

for ng_price in [4.42, 8]:
    model0.ng_price.fix(ng_price)
    model1.ng_price.fix(ng_price)
    model3p.ng_price.fix(ng_price)
    model4a.ng_price.fix(ng_price)
    model5p.ng_price.fix(ng_price)

    res = solver_obj.solve(model0)
    res = solver_obj.solve(model1)
    res = solver_obj.solve(model3p)
    res = solver_obj.solve(model4a)
    res = solver_obj.solve(model5p)

    capital[ng_price] = np.array([
        model0_fixed_cap/650, 
        model1_fixed_cap/650, 
        model3_fixed_cap/650, 
        model4_fixed_cap/650, 
        model5_fixed_cap/712,
    ])
    fixed_om[ng_price] = np.array([
        model0_fixed_om/650, 
        model1_fixed_om/650, 
        model3_fixed_om/650, 
        model4_fixed_om/650, 
        model5_fixed_om/712,
    ])
    fuel[ng_price] = np.array([
        pyo.value((model0.ng_price/4.42)*model0.fuel_cost)/650,
        pyo.value((model1.ng_price/4.42)*model1.fuel_cost)/650,
        pyo.value((model3p.ng_price/4.42)*model3p.fuel_cost)/650,
        pyo.value((model4a.ng_price/4.42)*model4a.fuel_cost)/650,
        pyo.value((model5p.ng_price/4.42)*model5p.fuel_cost)/712,
    ])
    other[ng_price] = np.array([
        pyo.value(model0.other_cost)/650,
        pyo.value(model1.other_cost)/650,
        pyo.value(model3p.other_cost)/650,
        pyo.value(model4a.other_cost)/650,
        pyo.value(model5p.other_cost)/712,
    ])
    
    
w = 0.4
ng_prices = [4.42, 8]
fig, ax = plt.subplots()
plt.grid(axis='y', zorder=0)
idx = np.arange(len(cases))

for i, ng_price in enumerate(ng_prices):
    if i == 0:
        l = "A"
    else:
        l = "B"
    b1 = plt.bar(idx - 0.2 + i*w, capital[ng_price], width=w, label="Capital", zorder=3, color='blue')
    b2 = plt.bar(idx - 0.2 + i*w, fixed_om[ng_price], width=w, label="Fixed O&M", bottom=capital[ng_price], zorder=3, color='c')
    plt.bar(idx - 0.2 + i*w, fuel[ng_price], width=w, label="Fuel", bottom=capital[ng_price]+fixed_om[ng_price], zorder=3, color='darkorange')
    b = plt.bar(idx - 0.2 + i*w, other[ng_price], width=w, label="Variable O&M", bottom=capital[ng_price]+fixed_om[ng_price]+fuel[ng_price], zorder=3, color='m')
    plt.bar_label(b, labels=[l]*len(cases))
plt.xlabel("Natural Gas Price (\$/million BTU): A) 4.42, B) 8.0")
plt.text(-0.5, 107.5, "Capacity Factors:\n  100% Power, 0% Hydrogen")
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc=1)
plt.xticks(idx, labels=cases)
plt.ylim(0, 120)
plt.ylabel("Cost of Electricity ($/MWhr)")
plt.savefig("power_bar.png", dpi=160, bbox_inches="tight")
plt.show()


### Hydrogen Only

In [ ]:
model3.profit.unfix()
model3.net_power.unfix()
model3.net_power.setub(0)
model3.h2_prod.fix(5)
model3.ng_price.fix(4.42)
model3.el_price.fix(71.70)

model4b.profit.unfix()
model4b.h2_prod.fix(5)
model4b.ng_price.fix(4.42)
model4b.el_price.fix(71.70)

model5.profit.unfix()
model5.net_power.unfix()
model5.net_power.setub(0)
model5.h2_prod.fix(5)
model5.ng_price.fix(4.42)
model5.el_price.fix(71.70)

model6.profit.unfix()
model6.h2_prod.fix(5)
model6.el_price.fix(71.70)

In [ ]:
cases = ["NGCC+SOEC", "rSOC", "SOFC+SOEC", "SOEC"]

capital = {}
fixed_om = {}
fuel = {}
elec = {}
other = {}

for ng_price in [4.42, 8]:
    for el_price in [30, 71.70, 100]:
        model3.ng_price.fix(ng_price)
        model4b.ng_price.fix(ng_price)
        model5.ng_price.fix(ng_price)
        model3.el_price.fix(el_price)
        model4b.el_price.fix(el_price)
        model5.el_price.fix(el_price)
        model6.el_price.fix(el_price)

        res = solver_obj.solve(model3)
        res = solver_obj.solve(model4b)
        res = solver_obj.solve(model5)
        res = solver_obj.solve(model6)

        capital[(ng_price, el_price)] = np.array([
            model3_fixed_cap/(5*60*60), 
            model4_fixed_cap/(5*60*60), 
            model5_fixed_cap/(5*60*60),
            model6_fixed_cap/(5*60*60),
        ])
        fixed_om[(ng_price, el_price)] = np.array([
            model3_fixed_om/(5*60*60), 
            model4_fixed_om/(5*60*60), 
            model5_fixed_om/(5*60*60),
            model6_fixed_om/(5*60*60),
        ])
        fuel[(ng_price, el_price)] = np.array([
            pyo.value((model3.ng_price/4.42)*model3.fuel_cost)/(5*60*60),
            pyo.value((model4b.ng_price/4.42)*model4b.fuel_cost)/(5*60*60),
            pyo.value((model5.ng_price/4.42)*model5.fuel_cost)/(5*60*60),
            0,
        ])
        elec[(ng_price, el_price)] = np.array([
            pyo.value(-model3.el_price*model3.net_power)/(5*60*60),
            pyo.value((model4b.el_price/30)*model4b.electricity_cost)/(5*60*60),
            pyo.value(-model5.el_price*model5.net_power)/(5*60*60),
            pyo.value((model6.el_price/30)*model6.electricity_cost)/(5*60*60),
        ])
        other[(ng_price, el_price)] = np.array([
            pyo.value(model3.other_cost)/(5*60*60),
            pyo.value(model4b.other_cost)/(5*60*60),
            pyo.value(model5.other_cost)/(5*60*60),
            pyo.value(model6.other_cost)/(5*60*60),
        ])

w = 0.15
ng_prices = [4.42, 8]
el_prices = [30, 71.70, 100]
fig, ax = plt.subplots()
fig.set_figwidth(7)
plt.grid(axis='y', zorder=0)
idx = np.arange(len(cases))
ng_price = 4.42
el_price = 30
i = 0
for ng_price in ng_prices:
    for el_price in el_prices:
        plt.bar(
            idx - 3*w + i*w, 
            capital[(ng_price, el_price)], 
            width=w, 
            label="Capital", 
            zorder=3,
            color='b',
        )
        plt.bar(
            idx - 3*w + i*w, 
            fixed_om[(ng_price, el_price)], 
            width=w,label="Fixed O&M", 
            bottom=capital[(ng_price, el_price)], 
            zorder=3,
            color='c',
        )
        plt.bar(
            idx - 3*w + i*w, 
            fuel[(ng_price, el_price)], 
            width=w, 
            label="Fuel", 
            bottom=capital[(ng_price, el_price)]+fixed_om[(ng_price, el_price)], 
            zorder=3,
            color='darkorange',
        )
        plt.bar(
            idx - 3*w + i*w, 
            elec[(ng_price, el_price)], 
            width=w, 
            label="Electricity", 
            bottom=capital[(ng_price, el_price)]+fixed_om[(ng_price, el_price)]+fuel[(ng_price, el_price)], 
            zorder=3,
            color='green'
        )
        b = plt.bar(
            idx - 3*w + i*w, 
            other[(ng_price, el_price)], 
            width=w, 
            label="Variable O&M", 
            bottom=capital[(ng_price, el_price)]+fixed_om[(ng_price, el_price)]+fuel[(ng_price, el_price)]+elec[(ng_price, el_price)], 
            zorder=3,
            color='m',
        )
        plt.bar_label(b, [{0:"A1", 1:"A2", 2:"A3", 3:"B1", 4:"B2", 5:"B3"}[i]]*len(cases))
        i += 1
plt.xlabel("Natural Gas Price (\$/million BTU): A) 4.42, B) 8.0\nElectricity Price (\$/MWhr): 1) 30.0, 2) 71.7, 3) 100.0")
plt.text(-0.65, 4.3, "Capacity Factors: \n  0% Power, 100% H$_2$")
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc="upper right", bbox_to_anchor=(0.775, 1))
plt.xticks(idx, labels=cases)
plt.ylim(0, 5.5)
plt.ylabel("Cost of Hydrogen ($/kg)")
plt.savefig("hydrogen_bar.png", dpi=160, bbox_inches="tight")
plt.show()

## Market LMP

For this we'll use the following abreviations:
1. e000 = ERCOT \\$0/ton Carbon Tax
2. e100 = ERCOT \\$100/ton Carbon Tax
3. c100 = CAISP \\$100/ton Carbon Tax

In [ ]:
# Read LMP Data
lmp_data = pd.read_csv("lmp2035.csv")

# Remake the models to undo any changes above

model0 = get_model0()
model1 = get_model1()
model3 = get_model3()
model3p = get_model3p()
model4a = get_model4a()
model4b = get_model4b()
model5 = get_model5()
model5p = get_model5p()
model6 = get_model6()

### Histogram Functions

In [ ]:
def profit_hist(dat, range=(None, None), nbins=20):
    if range[0] is None:
        range = (min(dat), range[1])
    if range[1] is None:
        range = (range[0], max(dat))
    plt.hist(dat, nbins, weights=np.ones(len(dat)) / len(dat), range=range)
    plt.xlabel("Net profit ($1000/hr)")
    plt.ylabel("Portion of time in bin")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    plt.show()
    
def power_hist(dat, range=(None, None), nbins=20):
    if range[0] is None:
        range = (min(dat), range[1])
    if range[1] is None:
        range = (range[0], max(dat))
    plt.hist(dat, nbins, weights=np.ones(len(dat)) / len(dat), range=range)
    plt.xlabel("Net Power (MW)")
    plt.ylabel("Portion of time in bin")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    plt.show()

def h2_hist(dat, range=(None, None), nbins=20):
    if range[0] is None:
        range = (min(dat), range[1])
    if range[1] is None:
        range = (range[0], max(dat))
    plt.hist(dat, 20, weights=np.ones(len(dat)) / len(dat), range=range)
    plt.xlabel("Hydrogen Production (kg/hr)")
    plt.ylabel("Portion of time in bin")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    plt.show()  

In [ ]:
print(lmp_data["CAISO_100"].mean())

### Select Data Set

In [ ]:
print(list(lmp_data.keys()))
lmp_data_set = "CAISO_100"

### Case 0 NGCC

This section calculates the optimal steady state operating point for Case 0 (NGCC) using the CAISO $100 Carbon Tax for 2035 data.  This assumes that there are not costs asssociated with startup and shutdown and the system can ramp instantly.

In [ ]:
solver_obj = pyo.SolverFactory("ipopt")

ng_price = 4.42
h2_price = 2

dat_profit = [] # profit
dat_e = [] # power output vector
dat_fuel = []
dat_other = []
dat_rev_e = []

model0.ng_price.fix(ng_price)

profit = []
for el_price in lmp_data[lmp_data_set]:
    #model0.net_power.fix(650)
    model0.el_price.fix(el_price)
    res = solver_obj.solve(model0)
    if not res.solver.termination_condition == pyo.TerminationCondition.optimal:
        raise Exception("fail")
    if pyo.value(model0.profit) > pyo.value(-model0.fixed_costs):
        dat_profit.append(pyo.value(model0.profit/1000))
        dat_e.append(pyo.value(model0.net_power))
        dat_fuel.append(pyo.value((model0.ng_price/4.42)*model0.fuel_cost))
        dat_other.append(pyo.value(model0.other_cost))
        dat_rev_e.append(pyo.value(model0.el_price*model0.net_power))
    else:
        dat_profit.append(pyo.value(-model0.fixed_costs/1000))
        dat_e.append(0)
        dat_fuel.append(0)
        dat_other.append(0)
        dat_rev_e.append(0)

In [ ]:
dat_zip = zip(dat_e, dat_fuel, dat_other, dat_rev_e, [p*1000 for p in dat_profit])
df = pd.DataFrame(dat_zip, columns=["Net Power (MW)", "Fuel Cost ($/hr)", "Other Cost ($/hr)", "Revenue ($/hr)", "Profit ($/hr)"])
df.to_csv("_".join(["c0", lmp_data_set, "opt.csv"]))

In [ ]:
c0_c100_profit = sum(p for p in dat_profit)
c0_c100_annual_power = sum(p for p in dat_e)

c0_c100_profit_vec = list(dat_profit)
c0_c100_power_vec = list(dat_e)

t_off = 0
t_power = 0
for p in c0_c100_power_vec:
    if p < 200:
        t_off += 1
    if p > 200:
        t_power += 1
        
c0_c100_pct_time_off = t_off/len(c0_c100_power_vec)*100
c0_c100_pct_time_power = t_power/len(c0_c100_power_vec)*100

In [ ]:
print(c0_c100_profit/1e3)
print(c0_c100_annual_power/1e6)
print(c0_c100_pct_time_off)
print(c0_c100_pct_time_power)
print(c0_c100_pct_time_off + c0_c100_pct_time_power)
print(c0_c100_profit_vec)

### Case 1 SOFC

This section calculates the optimal steady state operating point for Case 0 (SOFC) using the CAISO $100 Carbon Tax for 2035 data.  This assumes that there are not costs asssociated with startup and shutdown and the system can ramp instantly.

In [ ]:
solver_obj = pyo.SolverFactory("ipopt")

ng_price = 4.42
h2_price = 2

dat_profit = [] # profit
dat_e = [] # power output vector
dat_fuel = []
dat_other = []
dat_rev_e = []

model1.ng_price.fix(ng_price)

profit = []
for el_price in lmp_data[lmp_data_set]:
    model1.el_price.fix(el_price)
    res = solver_obj.solve(model1)
    if not res.solver.termination_condition == pyo.TerminationCondition.optimal:
        raise Exception("fail")
    if pyo.value(model1.profit) > pyo.value(-model1.fixed_costs):
        dat_profit.append(pyo.value(model1.profit/1000))
        dat_e.append(pyo.value(model1.net_power))
        dat_fuel.append(pyo.value((model1.ng_price/4.42)*model0.fuel_cost))
        dat_other.append(pyo.value(model1.other_cost))
        dat_rev_e.append(pyo.value(model1.el_price*model0.net_power))
    else:
        dat_profit.append(pyo.value(-model1.fixed_costs/1000))
        dat_e.append(0)
        dat_fuel.append(0)
        dat_other.append(0)
        dat_rev_e.append(0)

In [ ]:
dat_zip = zip(dat_e, dat_fuel, dat_other, dat_rev_e, [p*1000 for p in dat_profit])
df = pd.DataFrame(dat_zip, columns=["Net Power (MW)", "Fuel Cost ($/hr)", "Other Cost ($/hr)", "Revenue ($/hr)", "Profit ($/hr)"])
df.to_csv("_".join(["c1", lmp_data_set, "opt.csv"]))

In [ ]:
c1_c100_profit = sum(p for p in dat_profit)
c1_c100_annual_power = sum(p for p in dat_e)

c1_c100_profit_vec = dat_profit
c1_c100_power_vec = dat_e

t_off = 0
t_power = 0
for p in c1_c100_power_vec:
    if p < 200:
        t_off += 1
    if p > 200:
        t_power += 1
        
c1_c100_pct_time_off = t_off/len(c1_c100_power_vec)*100
c1_c100_pct_time_power = t_power/len(c1_c100_power_vec)*100

In [ ]:
print(c1_c100_profit/1e3)
print(c1_c100_annual_power/1e6)
print(c1_c100_pct_time_off)
print(c1_c100_pct_time_power)
print(c1_c100_pct_time_off + c1_c100_pct_time_power)

### Case 3 NGCC + SOEC

In [ ]:
solver_obj = pyo.SolverFactory("ipopt")

ng_price = 4.42
h2_price = 2

dat_profit = [] # profit
dat_h2 = [] # hydrogen output vector
dat_e = [] # power output vector
dat_fuel = []
dat_other = []
dat_rev_e = []
dat_rev_h = []
dat_fixed = []
dat_lmp = []

model3.ng_price.fix(ng_price)
model3p.ng_price.fix(ng_price)
model3.h2_price.fix(h2_price)

model3.net_power.setlb(None)
model3.net_power.setub(None)


profit = []
for el_price in lmp_data[lmp_data_set]:
    dat_lmp.append(el_price)
    dat_fixed.append(pyo.value(model3.fixed_costs))
    model3.el_price.fix(el_price)
    model3p.el_price.fix(el_price)
    res = solver_obj.solve(model3)
    if not res.solver.termination_condition == pyo.TerminationCondition.optimal:
        raise Exception("fail")
    res = solver_obj.solve(model3p)
    if not res.solver.termination_condition == pyo.TerminationCondition.optimal:
        raise Exception("fail")
    if pyo.value(model3.profit) > pyo.value(model3p.profit) and pyo.value(model3.profit) > pyo.value(-model3.fixed_costs):
        dat_profit.append(pyo.value(model3.profit/1000))
        dat_h2.append(pyo.value(model3.h2_prod*3600))
        dat_e.append(pyo.value(model3.net_power))
        dat_fuel.append(pyo.value(model3.fuel_cost))
        dat_other.append(pyo.value(model3.other_cost))
        dat_rev_e.append(pyo.value(model3.net_power*model3.el_price))
        dat_rev_h.append(pyo.value(model3.h2_prod*3600*model3.h2_price))
    elif pyo.value(model3p.profit) > pyo.value(-model3.fixed_costs):  
        dat_profit.append(pyo.value(model3p.profit/1000))
        dat_h2.append(0)
        dat_e.append(pyo.value(model3p.net_power))
        dat_fuel.append(pyo.value(model3p.fuel_cost))
        dat_other.append(pyo.value(model3p.other_cost))
        dat_rev_e.append(pyo.value(model3p.net_power*model3.el_price))
        dat_rev_h.append(0)
    else:
        dat_profit.append(pyo.value(-model3.fixed_costs/1000))
        dat_h2.append(0)
        dat_e.append(0)
        dat_fuel.append(0)
        dat_other.append(0)
        dat_rev_e.append(0)
        dat_rev_h.append(0)
        

In [ ]:
dat_zip = zip(dat_lmp, dat_fixed, dat_e, dat_fuel, dat_other, dat_rev_e, dat_rev_h, [p*1000 for p in dat_profit])
df = pd.DataFrame(dat_zip, columns=["LMP ($/MWh)", "Fixed Costs ($/hr)", "Net Power (MW)", "Fuel Cost ($/hr)", "Other Cost ($/hr)", "Power Revenue ($/hr)", "Hydrogen Revenue ($/hr)", "Profit ($/hr)"])
df.to_csv("_".join(["c3", lmp_data_set, "opt.csv"]))

In [ ]:
c3_c100_profit = sum(p for p in dat_profit)
c3_c100_annual_power = sum(p for p in dat_e)
c3_c100_annual_h2 = sum(p for p in dat_h2)

c3_c100_profit_vec = dat_profit
c3_c100_power_vec = dat_e
c3_c100_h2_vec = dat_h2

t_off = 0
t_power = 0
t_h2 = 0
for p, h in zip(c3_c100_power_vec, c3_c100_h2_vec):
    if p < 200 and h < 7500:
        t_off += 1
    if p > 200:
        t_power += 1
    if h > 7500:
        t_h2 += 1
        
c3_c100_pct_time_off = t_off/len(c3_c100_power_vec)*100
c3_c100_pct_time_power = t_power/len(c3_c100_power_vec)*100
c3_c100_pct_time_h2 = t_h2/len(c3_c100_power_vec)*100

In [ ]:
print(f"Annual Profit: {c3_c100_profit/1e3} million dollars")
print(f"Annual Power: {c3_c100_annual_power/1e6} million MWh")
print(f"Annual H2: {c3_c100_annual_h2/1e6} million kg")
print(c3_c100_pct_time_off)
print(c3_c100_pct_time_power)
print(c3_c100_pct_time_h2)
print(c3_c100_pct_time_off + c3_c100_pct_time_power + c3_c100_pct_time_h2)

In [ ]:
profit_hist(c3_c100_profit_vec, range=(None, 80))

In [ ]:
power_hist(c3_c100_power_vec)

In [ ]:
h2_hist(c3_c100_h2_vec)

In [ ]:
dat = sorted(zip(lmp_data[lmp_data_set], c3_c100_profit_vec), key=lambda x:x[0])
plt.plot([x[0] for x in dat], [x[1] for x in dat])
plt.xlabel("LMP ($/MWh)")
plt.ylabel("Profit ($1000/hr)")
plt.show()

dat = sorted(zip(lmp_data[lmp_data_set], c3_c100_power_vec), key=lambda x:x[0])
plt.plot([x[0] for x in dat], [x[1] for x in dat])
plt.xlabel("LMP ($/MWh)")
plt.ylabel("Power (MW)")
plt.show()

dat = sorted(zip(lmp_data[lmp_data_set], c3_c100_h2_vec), key=lambda x:x[0])
plt.plot([x[0] for x in dat], [x[1] for x in dat])
plt.xlabel("LMP ($/MWh)")
plt.ylabel("Hydrogen (kg/hr)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,5))
ax1 = host_subplot(111, axes_class=AA.Axes)
ax1.set_ylabel("LMP ($/MWh)")
plt.subplots_adjust(right=0.75)
ax2 = ax1.twinx()
ax3 = ax1.twinx()
ax2.axis["right"] = ax3.get_grid_helper().new_fixed_axis(loc="right", axes=ax2,offset=(0, 0))
ax2.set_ylabel("Power (MW)")
ax3.axis["right"] = ax3.get_grid_helper().new_fixed_axis(loc="right", axes=ax3,offset=(70, 0))
ax3.set_ylabel("Hydrogen (kg/hr)")

days = 30
day1 = 0

s1 = ax1.plot([i/24 for i in range(days*24)], lmp_data[lmp_data_set][day1*24:days*24 + day1*24], label="LMP", color="black")
s2 = ax2.plot([i/24 for i in range(days*24)], c3_c100_power_vec[day1*24:days*24+day1*24], label="power", color="g")
s3 = ax3.plot([i/24 for i in range(days*24)], c3_c100_h2_vec[day1*24:days*24+day1*24], label="power", color="r")
plt.xlabel("Day")
s = s1 + s2 + s3
ax1.legend(s, ["LMP", "power", "hydrogen"], loc=(0.01, 0.01))
plt.show()

### Case 4 rSOC

In [ ]:
solver_obj = pyo.SolverFactory("ipopt")

ng_price = 4.42
h2_price = 2

dat_profit = [] # profit
dat_h2 = [] # hydrogen output vector
dat_e = [] # power output vector

model4a.ng_price.fix(ng_price)
model4b.ng_price.fix(ng_price)
model4b.h2_price.fix(h2_price)

profit = []
for el_price in lmp_data[lmp_data_set]:
    model4a.el_price.fix(el_price)
    model4b.el_price.fix(el_price)
    res = solver_obj.solve(model4a)
    if not res.solver.termination_condition == pyo.TerminationCondition.optimal:
        raise Exception("fail")
    res = solver_obj.solve(model4b)
    if not res.solver.termination_condition == pyo.TerminationCondition.optimal:
        raise Exception("fail")
    if pyo.value(model4a.profit) > pyo.value(model4b.profit) and pyo.value(model4a.profit) > pyo.value(-model4a.fixed_costs):
        dat_profit.append(pyo.value(model4a.profit/1000))
        dat_h2.append(0)
        dat_e.append(pyo.value(model4a.net_power)) 
    elif pyo.value(model4b.profit) > pyo.value(-model4a.fixed_costs):  
        dat_profit.append(pyo.value(model4b.profit/1000))
        dat_h2.append(pyo.value(model4b.h2_prod*3600))
        dat_e.append(pyo.value(-model4b.electricity_cost/30.0)) 
    else:
        dat_profit.append(pyo.value(-model4a.fixed_costs/1000))
        dat_h2.append(0)
        dat_e.append(0)   

In [ ]:
c4_c100_profit = sum(p for p in dat_profit)
c4_c100_annual_power = sum(p for p in dat_e)
c4_c100_annual_h2 = sum(p for p in dat_h2)

c4_c100_profit_vec = dat_profit
c4_c100_power_vec = dat_e
c4_c100_h2_vec = dat_h2

t_off = 0
t_power = 0
t_h2 = 0
for p, h in zip(c4_c100_power_vec, c4_c100_h2_vec):
    if p < 200 and h < 7500:
        t_off += 1
    if p > 200:
        t_power += 1
    if h > 7500:
        t_h2 += 1
        
c4_c100_pct_time_off = t_off/len(c4_c100_power_vec)*100
c4_c100_pct_time_power = t_power/len(c4_c100_power_vec)*100
c4_c100_pct_time_h2 = t_h2/len(c4_c100_power_vec)*100

In [ ]:
print(f"Annual Profit: {c4_c100_profit/1e3} million dollars")
print(f"Annual Power: {c4_c100_annual_power/1e6} million MWh")
print(f"Annual H2: {c4_c100_annual_h2/1e6} million kg")
print(c4_c100_pct_time_off)
print(c4_c100_pct_time_power)
print(c4_c100_pct_time_h2)
print(c4_c100_pct_time_off + c4_c100_pct_time_power + c4_c100_pct_time_h2)

### Case 5 SOFC + SOEC

In [ ]:
solver_obj = pyo.SolverFactory("ipopt")

ng_price = 4.42
h2_price = 2

dat_profit = [] # profit
dat_h2 = [] # hydrogen output vector
dat_e = [] # power output vector
dat_fuel = []
dat_other = []
dat_rev_e = []
dat_rev_h = []

model5.ng_price.fix(ng_price)
model5p.ng_price.fix(ng_price)
model5.h2_price.fix(h2_price)

model5.net_power.setlb(None)
model5.net_power.setub(None)


profit = []
for el_price in lmp_data[lmp_data_set]:
    model5.el_price.fix(el_price)
    model5p.el_price.fix(el_price)
    res = solver_obj.solve(model5)
    if not res.solver.termination_condition == pyo.TerminationCondition.optimal:
        raise Exception("fail")
    res = solver_obj.solve(model5p)
    if not res.solver.termination_condition == pyo.TerminationCondition.optimal:
        raise Exception("fail")
    if pyo.value(model5.profit) > pyo.value(model5p.profit) and pyo.value(model5.profit) > pyo.value(-model5.fixed_costs):
        dat_profit.append(pyo.value(model5.profit/1000))
        dat_h2.append(pyo.value(model5.h2_prod*3600))
        dat_e.append(pyo.value(model5.net_power))
        dat_fuel.append(pyo.value(model5.fuel_cost))
        dat_other.append(pyo.value(model5.other_cost))
        dat_rev_e.append(pyo.value(model5.net_power*model5.el_price))
        dat_rev_h.append(pyo.value(model5.h2_prod*3600*model5.h2_price))
    elif pyo.value(model5p.profit) > pyo.value(-model5.fixed_costs):  
        dat_profit.append(pyo.value(model5p.profit/1000))
        dat_h2.append(0)
        dat_e.append(pyo.value(model5p.net_power))
        dat_fuel.append(pyo.value(model5p.fuel_cost))
        dat_other.append(pyo.value(model5p.other_cost))
        dat_rev_e.append(pyo.value(model5p.net_power*model5.el_price))
        dat_rev_h.append(0)
    else:
        dat_profit.append(pyo.value(-model5.fixed_costs/1000))
        dat_fuel.append(0)
        dat_other.append(0)
        dat_rev_e.append(0)
        dat_rev_h.append(0)
        dat_e.append(0)
        dat_h2.append(0)
        

In [ ]:
dat_zip = zip(dat_e, dat_fuel, dat_other, dat_rev_e, dat_rev_h, [p*1000 for p in dat_profit])
df = pd.DataFrame(dat_zip, columns=["Net Power (MW)", "Fuel Cost ($/hr)", "Other Cost ($/hr)", "Power Revenue ($/hr)", "Hydrogen Revenue ($/hr)", "Profit ($/hr)"])
df.to_csv("_".join(["c5", lmp_data_set, "opt.csv"]))

In [ ]:
c5_c100_profit = sum(p for p in dat_profit)
c5_c100_annual_power = sum(p for p in dat_e)
c5_c100_annual_h2 = sum(p for p in dat_h2)

c5_c100_profit_vec = dat_profit
c5_c100_power_vec = dat_e
c5_c100_h2_vec = dat_h2

t_off = 0
t_power = 0
t_h2 = 0
for p, h in zip(c5_c100_power_vec, c5_c100_h2_vec):
    if p < 200 and h < 7500:
        t_off += 1
    if p > 200:
        t_power += 1
    if h > 7500:
        t_h2 += 1
        
c5_c100_pct_time_off = t_off/len(c5_c100_power_vec)*100
c5_c100_pct_time_power = t_power/len(c5_c100_power_vec)*100
c5_c100_pct_time_h2 = t_h2/len(c5_c100_power_vec)*100

In [ ]:
print(f"Annual Profit: {c5_c100_profit/1e3} million dollars")
print(f"Annual Power: {c5_c100_annual_power/1e6} million MWh")
print(f"Annual H2: {c5_c100_annual_h2/1e6} million kg")
print(c5_c100_pct_time_off)
print(c5_c100_pct_time_power)
print(c5_c100_pct_time_h2)
print(c5_c100_pct_time_off + c5_c100_pct_time_power + c5_c100_pct_time_h2)

In [ ]:
profit_hist(c5_c100_profit_vec, range=(None, 80))

In [ ]:
power_hist(c5_c100_power_vec)

In [ ]:
h2_hist(c5_c100_h2_vec)

In [ ]:
dat = sorted(zip(lmp_data[lmp_data_set], c5_c100_profit_vec), key=lambda x:x[0])
plt.plot([x[0] for x in dat], [x[1] for x in dat])
plt.xlabel("LMP ($/MWh)")
plt.ylabel("Profit ($1000/hr)")
plt.show()

dat = sorted(zip(lmp_data[lmp_data_set], c5_c100_power_vec), key=lambda x:x[0])
plt.plot([x[0] for x in dat], [x[1] for x in dat])
plt.xlabel("LMP ($/MWh)")
plt.ylabel("Power (MW)")
plt.show()

dat = sorted(zip(lmp_data[lmp_data_set], c5_c100_h2_vec), key=lambda x:x[0])
plt.plot([x[0] for x in dat], [x[1] for x in dat])
plt.xlabel("LMP ($/MWh)")
plt.ylabel("Hydrogen (kg/hr)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,5))
ax1 = host_subplot(111, axes_class=AA.Axes)
ax1.set_ylabel("LMP ($/MWh)")
plt.subplots_adjust(right=0.75)
ax2 = ax1.twinx()
ax2.axis["right"] = ax3.get_grid_helper().new_fixed_axis(loc="right", axes=ax2,offset=(0, 0))
ax2.set_ylabel("Power (MW)")
ax3 = ax1.twinx()
ax3.axis["right"] = ax3.get_grid_helper().new_fixed_axis(loc="right", axes=ax3,offset=(70, 0))
ax3.set_ylabel("Hydrogen (kg/hr)")

days = 30
day1 = 0

s1 = ax1.plot([i/24 for i in range(days*24)], lmp_data[lmp_data_set][day1*24:days*24 + day1*24], label="LMP", color="black")
s2 = ax2.plot([i/24 for i in range(days*24)], c5_c100_power_vec[day1*24:days*24+day1*24], label="power", color="g")
s3 = ax3.plot([i/24 for i in range(days*24)], c5_c100_h2_vec[day1*24:days*24+day1*24], label="power", color="r")
plt.xlabel("Day")
s = s1 + s2 + s3
ax1.legend(s, ["LMP", "power", "hydrogen"], loc=(0.01, 0.01))
plt.show()

### Case 6 SOEC

In [ ]:
solver_obj = pyo.SolverFactory("ipopt")

ng_price = 4.42
h2_price = 2

dat_profit = [] # profit
dat_h2 = [] # hydrogen output vector
dat_e = [] # power output vector
dat_other = []
dat_e_cost = [] # power output vector

model6.h2_price.fix(h2_price)

profit = []
for el_price in lmp_data[lmp_data_set]:
    model6.el_price.fix(el_price)
    res = solver_obj.solve(model6)
    if not res.solver.termination_condition == pyo.TerminationCondition.optimal:
        raise Exception("fail")
    elif pyo.value(model6.profit) > pyo.value(-model6.fixed_costs):  
        dat_profit.append(pyo.value(model6.profit/1000))
        dat_h2.append(pyo.value(model6.h2_prod*3600))
        dat_e.append(pyo.value(-model6.electricity_cost/30.0))
        dat_e_cost.append(pyo.value((model6.el_price/30)*model6.electricity_cost))
        dat_other.append(pyo.value(model6.other_cost))
        dat_rev_h.append(pyo.value(model6.h2_prod*3600*model6.h2_price))
        
    else:
        dat_profit.append(pyo.value(-model6.fixed_costs/1000))
        dat_h2.append(0)
        dat_e.append(0)
        dat_e_cost.append(0)
        dat_other.append(0)
        dat_rev_h.append(0)

In [ ]:
dat_zip = zip(dat_e, dat_e_cost, dat_other, dat_h2, dat_rev_h, [p*1000 for p in dat_profit])
df = pd.DataFrame(dat_zip, columns=[
    "Net Power (MW)", 
    "Electricity Cost ($/hr)", 
    "Other Cost ($/hr)",
    "Hydrgen (kg/hr)",
    "Hydrogen Revenue ($/hr)", 
    "Profit ($/hr)"
])
df.to_csv("_".join(["c6", lmp_data_set, "opt.csv"]))

In [ ]:
c6_c100_profit = sum(p for p in dat_profit)
c6_c100_annual_power = sum(p for p in dat_e)
c6_c100_annual_h2 = sum(p for p in dat_h2)

c6_c100_profit_vec = dat_profit
c6_c100_power_vec = dat_e
c6_c100_h2_vec = dat_h2

t_off = 0
t_power = 0
t_h2 = 0
for p, h in zip(c6_c100_power_vec, c6_c100_h2_vec):
    if h < 100:
        t_off += 1
    if h > 100:
        t_h2 += 1
        
c6_c100_pct_time_off = t_off/len(c6_c100_power_vec)*100
c6_c100_pct_time_power = t_power/len(c6_c100_power_vec)*100
c6_c100_pct_time_h2 = t_h2/len(c6_c100_power_vec)*100

In [ ]:
print(f"Annual Profit: {c6_c100_profit/1e3} million dollars")
print(f"Annual Power: {c6_c100_annual_power/1e6} million MWh")
print(f"Annual H2: {c6_c100_annual_h2/1e6} million kg")
print(c6_c100_pct_time_off)
print(c6_c100_pct_time_power)
print(c6_c100_pct_time_h2)
print(c6_c100_pct_time_off + c6_c100_pct_time_power + c6_c100_pct_time_h2)